In [ ]:
import tensorflow as tf
import cv2
import numpy as np
from tensorflow.keras.layers import Dropout, Flatten, Dense, MaxPooling2D, Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
import os


In [ ]:
fldr = "E:/pedestrian detector/train"

In [ ]:
images = []
labels = []

for filename in os.listdir(fldr):
    total = os.path.join(fldr, filename)
    image = cv2.imread(total)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (48, 48))
    images.append(image)
    label = 1 if "pedestrian" in filename else 0
    labels.append(label)

In [ ]:
images_f = np.array(images) / 255.0
labels_f = np.array(labels)


In [ ]:
np.save(fldr + 'images.npy', images_f)
np.save(fldr + 'labels.npy', labels_f)


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(images_f, labels_f, test_size=0.25)


In [ ]:
def Convolution(input_tensor, filters):
    x = Conv2D(filters=filters, kernel_size=(3, 3), padding="same", strides=(1, 1))(input_tensor)
    x = Dropout(0.1)(x)
    x = Activation('relu')(x)
    return x

In [ ]:

def pedestrian_detection_model(input_shape):
    inputs = Input((input_shape))
    conv_1 = Convolution(inputs, 32)
    maxp_1 = MaxPooling2D(pool_size=(2, 2))(conv_1)
    conv_2 = Convolution(maxp_1, 64)
    maxp_2 = MaxPooling2D(pool_size=(2, 2))(conv_2)
    conv_3 = Convolution(maxp_2, 128)
    maxp_3 = MaxPooling2D(pool_size=(2, 2))(conv_3)
    conv_4 = Convolution(maxp_3, 256)
    maxp_4 = MaxPooling2D(pool_size=(2, 2))(conv_4)
    flatten = Flatten()(maxp_4)
    dense_1 = Dense(64, activation='relu')(flatten)
    drop_1 = Dropout(0.2)(dense_1)
    output = Dense(1, activation='sigmoid', name='pedestrian_out')(drop_1)

    model = Model(inputs=[inputs], outputs=[output])
    model.compile(loss="binary_crossentropy", optimizer=Adam(), metrics=["accuracy"])
    return model

In [ ]:
pedestrian_model = pedestrian_detection_model((48, 48, 3))
pedestrian_model.summary()

In [ ]:
pedestrian_filename = 'modelPedestrianDetection.h5'

In [ ]:
checkpoint_pedestrian = ModelCheckpoint(pedestrian_filename, monitor='val_loss', verbose=1,
                                         save_best_only=True, save_weights_only=False, mode='auto', save_freq='epoch')
early_stop_pedestrian = EarlyStopping(patience=75, monitor='val_loss', restore_best_weights=True)
callback_list_pedestrian = [checkpoint_pedestrian, early_stop_pedestrian]


In [ ]:
history_pedestrian = pedestrian_model.fit(X_train, Y_train, batch_size=64, validation_data=(X_test, Y_test),
                                          epochs=500, callbacks=callback_list_pedestrian)
pedestrian_model.evaluate(X_test, Y_test)
